In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import numpy as np
import re
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns
import urllib.robotparser as urobot
from urllib.request import urlopen
from pylab import rcParams
import csv

%matplotlib inline

In [2]:
#Load both list_of_champs table & prospects'

df1 = pd.read_csv('../data/list_of_champs.csv')
df2 = pd.read_csv('../data/Top50_prospects.csv')
df2 = df2.dropna(axis=1)
df2

,Rank,Name,Weight Class,Record,Age,Fighting Style,Company,Country,Nickname
0,1,Vadim Nemkov,Light Heavyweight,11-2,27,Wrestling,Bellator,Russia,Unknown
1,2,Lance Palmer,Featherweight,22-3,32,Wrestling,PFL,USA,Unknown
2,3,A.J. McKee,Featherweight,16-0,25,Wrestling,Bellator,USA,Unknown
3,4,Natan Schulte,Lightweight,20-3-1,28,Jiu-Jitsu,PFL,Brazil,Russo
4,5,Darrion Caldwell,Featherweight,15-3,32,Wrestling,Bellator,USA,Unknown
5,6,Jiri Prochazka,Light Heavyweight,26-3-1,27,Boxing,UFC,Czech republic,Unknown
6,7,Valentin Moldavsky,Heavyweight,10-1,28,"Sambo, MMA",Bellator,Russia,Unknown
7,8,Abdul-Aziz Abdulvakhabov,Lightweight,18-2,31,Wrestling,ACA,Russia,Lion
8,9,Tomasz Narkun,Light Heavyweight,18-3,30,Jiu-Jitsu,KSW,Poland,Giraffe
9,10,Pedro Carvalho,Featherweight,11-4,24,Jiu-Jitsu,Bellator,Portugal,The Game


In [3]:
###Adjust df2 to match up with the list of champs before joining both tables###

df2 = pd.read_csv('../data/Top50_prospects.csv', usecols = [' Weight Class',
                                                            ' Name',
                                                            ' Record',
                                                            'Fighting Style',
                                                            'Country',
                                                            'Nickname'])
df2

,Name,Weight Class,Record,Fighting Style,Country,Nickname
0,Vadim Nemkov,Light Heavyweight,11-2,Wrestling,Russia,Unknown
1,Lance Palmer,Featherweight,22-3,Wrestling,USA,Unknown
2,A.J. McKee,Featherweight,16-0,Wrestling,USA,Unknown
3,Natan Schulte,Lightweight,20-3-1,Jiu-Jitsu,Brazil,Russo
4,Darrion Caldwell,Featherweight,15-3,Wrestling,USA,Unknown
5,Jiri Prochazka,Light Heavyweight,26-3-1,Boxing,Czech republic,Unknown
6,Valentin Moldavsky,Heavyweight,10-1,"Sambo, MMA",Russia,Unknown
7,Abdul-Aziz Abdulvakhabov,Lightweight,18-2,Wrestling,Russia,Lion
8,Tomasz Narkun,Light Heavyweight,18-3,Jiu-Jitsu,Poland,Giraffe
9,Pedro Carvalho,Featherweight,11-4,Jiu-Jitsu,Portugal,The Game


In [4]:
## Since df2 only showcases the top50 'MALE' MMA prospects, I'll add a 'Sex' colummn to clarify my analysis later on btw male prospects vs champs

df2['Sex'] = 'Male'
df2

,Name,Weight Class,Record,Fighting Style,Country,Nickname,Sex
0,Vadim Nemkov,Light Heavyweight,11-2,Wrestling,Russia,Unknown,Male
1,Lance Palmer,Featherweight,22-3,Wrestling,USA,Unknown,Male
2,A.J. McKee,Featherweight,16-0,Wrestling,USA,Unknown,Male
3,Natan Schulte,Lightweight,20-3-1,Jiu-Jitsu,Brazil,Russo,Male
4,Darrion Caldwell,Featherweight,15-3,Wrestling,USA,Unknown,Male
5,Jiri Prochazka,Light Heavyweight,26-3-1,Boxing,Czech republic,Unknown,Male
6,Valentin Moldavsky,Heavyweight,10-1,"Sambo, MMA",Russia,Unknown,Male
7,Abdul-Aziz Abdulvakhabov,Lightweight,18-2,Wrestling,Russia,Lion,Male
8,Tomasz Narkun,Light Heavyweight,18-3,Jiu-Jitsu,Poland,Giraffe,Male
9,Pedro Carvalho,Featherweight,11-4,Jiu-Jitsu,Portugal,The Game,Male


In [5]:
## Now, let's merge both tables = combined list

df3 = pd.merge(df1, df2, how = 'outer')
df3

,Weight Class,Name,Sex,Record,Fighting Style,Country,Nickname,Title Holder
0,Heavyweight\n,Mark Coleman\n,Male,16-10,Wrestling,USA,The Hammer,Former
1,Heavyweight\n,Maurice Smith\n,Male,14-14,Kickboxing,USA,Mo,Former
2,Heavyweight\n,Randy Couture\n,Male,19-11,Wrestling,USA,The Natural,Former
3,Heavyweight\n,Randy Couture\n,Male,19-11,Wrestling,USA,The Natural,Former
4,Heavyweight\n,Bas Rutten\n,Male,28-4,"Kickboxing, Wrestling",Netherlands,El Guapo (The Handsome One),Former
...,...,...,...,...,...,...,...,...
164,Light Heavyweight,Nasrudin Nasrudinov,Male,9-0,Wrestling,Russia,Unknown,NaN
165,Middleweight,Vladimir Mineev,Male,13-1-1,Kickboxing,Russia,Unknown,NaN
166,Bantamweight,Patrick Mix,Male,13-0,"Wrestling, Jiu-Jitsu",USA,No Love,NaN
167,Lightweight,Soren Bak,Male,13-1,"Wrestling, Thai Boxing",Unknown,The True Viking,NaN


In [6]:
df3.tail()

,Weight Class,Name,Sex,Record,Fighting Style,Country,Nickname,Title Holder
164,Light Heavyweight,Nasrudin Nasrudinov,Male,9-0,Wrestling,Russia,Unknown,NaN
165,Middleweight,Vladimir Mineev,Male,13-1-1,Kickboxing,Russia,Unknown,NaN
166,Bantamweight,Patrick Mix,Male,13-0,"Wrestling, Jiu-Jitsu",USA,No Love,NaN
167,Lightweight,Soren Bak,Male,13-1,"Wrestling, Thai Boxing",Unknown,The True Viking,NaN
168,Lightweight,Khusein Khaliev,Male,19-2,Wrestling,Russia,Captain,NaN


In [7]:
# Remove "\n" on both weight class, name columns

df3[' Weight Class'] = df3[' Weight Class'].str.strip()
df3.head()

,Weight Class,Name,Sex,Record,Fighting Style,Country,Nickname,Title Holder
0,Heavyweight,Mark Coleman\n,Male,16-10,Wrestling,USA,The Hammer,Former
1,Heavyweight,Maurice Smith\n,Male,14-14,Kickboxing,USA,Mo,Former
2,Heavyweight,Randy Couture\n,Male,19-11,Wrestling,USA,The Natural,Former
3,Heavyweight,Randy Couture\n,Male,19-11,Wrestling,USA,The Natural,Former
4,Heavyweight,Bas Rutten\n,Male,28-4,"Kickboxing, Wrestling",Netherlands,El Guapo (The Handsome One),Former


In [8]:
df3[' Name'] = df3[' Name'].str.strip()
df3.head()

,Weight Class,Name,Sex,Record,Fighting Style,Country,Nickname,Title Holder
0,Heavyweight,Mark Coleman,Male,16-10,Wrestling,USA,The Hammer,Former
1,Heavyweight,Maurice Smith,Male,14-14,Kickboxing,USA,Mo,Former
2,Heavyweight,Randy Couture,Male,19-11,Wrestling,USA,The Natural,Former
3,Heavyweight,Randy Couture,Male,19-11,Wrestling,USA,The Natural,Former
4,Heavyweight,Bas Rutten,Male,28-4,"Kickboxing, Wrestling",Netherlands,El Guapo (The Handsome One),Former


In [9]:
#Let's split the record column into two more descriptive 'Wins' & 'Losses' columns

split = lambda x:pd.Series([i for i in (x.split('-'))])

In [10]:
Record_df = df3[' Record'].apply(split)
Record_df

,0,1,2
0,16,10,NaN
1,14,14,NaN
2,19,11,NaN
3,19,11,NaN
4,28,4,NaN
...,...,...,...
164,9,0,NaN
165,13,1,1
166,13,0,NaN
167,13,1,NaN


In [11]:
Record_df.columns = ['Wins',
                     'Losses',
                     'Draws']

Record_df.head(2)

,Wins,Losses,Draws
0,16,10,NaN
1,14,14,NaN


In [12]:
Record_df = Record_df.drop(['Draws'], axis=1)

In [13]:
df4 = pd.concat([Record_df, df3], axis = 1)

In [14]:
# Reorder columns and remove extra space at the beginning of 'Weight Class' and Record columns
df4.columns = ['Wins', 'Losses', 'Weight Class', 'Name', 'Sex', 'Record',
       'Fighting Style', 'Country', 'Nickname', 'Title Holder']

In [15]:
df4[['Name', 'Weight Class', 'Wins', 'Losses', 'Fighting Style', 'Sex', 'Country', 'Nickname', 'Title Holder']]
df4 = df4[['Name', 'Weight Class', 'Wins', 'Losses', 'Fighting Style', 'Sex', 'Country', 'Nickname', 'Title Holder']]
df4

,Name,Weight Class,Wins,Losses,Fighting Style,Sex,Country,Nickname,Title Holder
0,Mark Coleman,Heavyweight,16,10,Wrestling,Male,USA,The Hammer,Former
1,Maurice Smith,Heavyweight,14,14,Kickboxing,Male,USA,Mo,Former
2,Randy Couture,Heavyweight,19,11,Wrestling,Male,USA,The Natural,Former
3,Randy Couture,Heavyweight,19,11,Wrestling,Male,USA,The Natural,Former
4,Bas Rutten,Heavyweight,28,4,"Kickboxing, Wrestling",Male,Netherlands,El Guapo (The Handsome One),Former
...,...,...,...,...,...,...,...,...,...
164,Nasrudin Nasrudinov,Light Heavyweight,9,0,Wrestling,Male,Russia,Unknown,NaN
165,Vladimir Mineev,Middleweight,13,1,Kickboxing,Male,Russia,Unknown,NaN
166,Patrick Mix,Bantamweight,13,0,"Wrestling, Jiu-Jitsu",Male,USA,No Love,NaN
167,Soren Bak,Lightweight,13,1,"Wrestling, Thai Boxing",Male,Unknown,The True Viking,NaN


In [16]:
#Split the name columns into first and last names

split = lambda x:pd.Series([i for i in (x.split(' '))])

In [17]:
Name_df = df4['Name'].apply(split)
Name_df

,0,1,2,3,4
0,Mark,Coleman,NaN,NaN,NaN
1,Maurice,Smith,NaN,NaN,NaN
2,Randy,Couture,NaN,NaN,NaN
3,Randy,Couture,NaN,NaN,NaN
4,Bas,Rutten,NaN,NaN,NaN
...,...,...,...,...,...
164,Nasrudin,Nasrudinov,NaN,NaN,NaN
165,Vladimir,Mineev,NaN,NaN,NaN
166,Patrick,Mix,NaN,NaN,NaN
167,Soren,Bak,NaN,NaN,NaN


In [18]:
Name_df.columns = ['First Name',
                   'Last Name',
                   'Interim Title Holder',
                   'n/a',
                   'n/a']

Name_df.head(2)

,First Name,Last Name,Interim Title Holder,n/a,n/a
0,Mark,Coleman,NaN,NaN,NaN
1,Maurice,Smith,NaN,NaN,NaN


In [19]:
Name_df = Name_df.drop(['n/a'], axis=1)
Name_df

,First Name,Last Name,Interim Title Holder
0,Mark,Coleman,NaN
1,Maurice,Smith,NaN
2,Randy,Couture,NaN
3,Randy,Couture,NaN
4,Bas,Rutten,NaN
...,...,...,...
164,Nasrudin,Nasrudinov,NaN
165,Vladimir,Mineev,NaN
166,Patrick,Mix,NaN
167,Soren,Bak,NaN


In [20]:
#remove "("

Name_df['Interim Title Holder'] = Name_df['Interim Title Holder'].str.strip('(')
Name_df.tail()

<ipython-input-20-c23243f857d4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Name_df['Interim Title Holder'] = Name_df['Interim Title Holder'].str.strip('(')


,First Name,Last Name,Interim Title Holder
164,Nasrudin,Nasrudinov,NaN
165,Vladimir,Mineev,NaN
166,Patrick,Mix,NaN
167,Soren,Bak,NaN
168,Khusein,Khaliev,NaN


In [21]:
df5 = pd.concat([Name_df, df4], axis = 1)
df5

,First Name,Last Name,Interim Title Holder,Name,Weight Class,Wins,Losses,Fighting Style,Sex,Country,Nickname,Title Holder
0,Mark,Coleman,NaN,Mark Coleman,Heavyweight,16,10,Wrestling,Male,USA,The Hammer,Former
1,Maurice,Smith,NaN,Maurice Smith,Heavyweight,14,14,Kickboxing,Male,USA,Mo,Former
2,Randy,Couture,NaN,Randy Couture,Heavyweight,19,11,Wrestling,Male,USA,The Natural,Former
3,Randy,Couture,NaN,Randy Couture,Heavyweight,19,11,Wrestling,Male,USA,The Natural,Former
4,Bas,Rutten,NaN,Bas Rutten,Heavyweight,28,4,"Kickboxing, Wrestling",Male,Netherlands,El Guapo (The Handsome One),Former
...,...,...,...,...,...,...,...,...,...,...,...,...
164,Nasrudin,Nasrudinov,NaN,Nasrudin Nasrudinov,Light Heavyweight,9,0,Wrestling,Male,Russia,Unknown,NaN
165,Vladimir,Mineev,NaN,Vladimir Mineev,Middleweight,13,1,Kickboxing,Male,Russia,Unknown,NaN
166,Patrick,Mix,NaN,Patrick Mix,Bantamweight,13,0,"Wrestling, Jiu-Jitsu",Male,USA,No Love,NaN
167,Soren,Bak,NaN,Soren Bak,Lightweight,13,1,"Wrestling, Thai Boxing",Male,Unknown,The True Viking,NaN


In [22]:
df5 = df5.drop(['Name'], axis=1)
df5

,First Name,Last Name,Interim Title Holder,Weight Class,Wins,Losses,Fighting Style,Sex,Country,Nickname,Title Holder
0,Mark,Coleman,NaN,Heavyweight,16,10,Wrestling,Male,USA,The Hammer,Former
1,Maurice,Smith,NaN,Heavyweight,14,14,Kickboxing,Male,USA,Mo,Former
2,Randy,Couture,NaN,Heavyweight,19,11,Wrestling,Male,USA,The Natural,Former
3,Randy,Couture,NaN,Heavyweight,19,11,Wrestling,Male,USA,The Natural,Former
4,Bas,Rutten,NaN,Heavyweight,28,4,"Kickboxing, Wrestling",Male,Netherlands,El Guapo (The Handsome One),Former
...,...,...,...,...,...,...,...,...,...,...,...
164,Nasrudin,Nasrudinov,NaN,Light Heavyweight,9,0,Wrestling,Male,Russia,Unknown,NaN
165,Vladimir,Mineev,NaN,Middleweight,13,1,Kickboxing,Male,Russia,Unknown,NaN
166,Patrick,Mix,NaN,Bantamweight,13,0,"Wrestling, Jiu-Jitsu",Male,USA,No Love,NaN
167,Soren,Bak,NaN,Lightweight,13,1,"Wrestling, Thai Boxing",Male,Unknown,The True Viking,NaN


In [23]:
# Reorder & record columns
df5.columns = ['First Name', 'Last Name', 'Interim Title Holder', 'Weight Class', 'Wins', 'Losses', 'Fighting Style', 'Sex', 'Country', 'Nickname', 'Title Holder']

In [24]:
df5[['First Name', 'Last Name', 'Weight Class', 'Wins', 'Losses', 'Fighting Style', 'Sex', 'Country', 'Nickname', 'Title Holder', 'Interim Title Holder']]
df5 = df5[['First Name', 'Last Name', 'Weight Class', 'Wins', 'Losses', 'Fighting Style', 'Sex', 'Country', 'Nickname', 'Title Holder', 'Interim Title Holder']]
df5

,First Name,Last Name,Weight Class,Wins,Losses,Fighting Style,Sex,Country,Nickname,Title Holder,Interim Title Holder
0,Mark,Coleman,Heavyweight,16,10,Wrestling,Male,USA,The Hammer,Former,NaN
1,Maurice,Smith,Heavyweight,14,14,Kickboxing,Male,USA,Mo,Former,NaN
2,Randy,Couture,Heavyweight,19,11,Wrestling,Male,USA,The Natural,Former,NaN
3,Randy,Couture,Heavyweight,19,11,Wrestling,Male,USA,The Natural,Former,NaN
4,Bas,Rutten,Heavyweight,28,4,"Kickboxing, Wrestling",Male,Netherlands,El Guapo (The Handsome One),Former,NaN
...,...,...,...,...,...,...,...,...,...,...,...
164,Nasrudin,Nasrudinov,Light Heavyweight,9,0,Wrestling,Male,Russia,Unknown,NaN,NaN
165,Vladimir,Mineev,Middleweight,13,1,Kickboxing,Male,Russia,Unknown,NaN,NaN
166,Patrick,Mix,Bantamweight,13,0,"Wrestling, Jiu-Jitsu",Male,USA,No Love,NaN,NaN
167,Soren,Bak,Lightweight,13,1,"Wrestling, Thai Boxing",Male,Unknown,The True Viking,NaN,NaN


In [25]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169 entries, 0 to 168
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   First Name            169 non-null    object
 1   Last Name             169 non-null    object
 2   Weight Class          169 non-null    object
 3   Wins                  169 non-null    object
 4   Losses                169 non-null    object
 5   Fighting Style        169 non-null    object
 6   Sex                   169 non-null    object
 7   Country               167 non-null    object
 8   Nickname              169 non-null    object
 9   Title Holder          119 non-null    object
 10  Interim Title Holder  15 non-null     object
dtypes: object(11)
memory usage: 15.8+ KB


In [26]:
df5.describe

<bound method NDFrame.describe of     First Name   Last Name       Weight Class Wins Losses  \
0         Mark     Coleman        Heavyweight   16     10   
1      Maurice       Smith        Heavyweight   14     14   
2        Randy     Couture        Heavyweight   19     11   
3        Randy     Couture        Heavyweight   19     11   
4          Bas      Rutten        Heavyweight   28     4    
..         ...         ...                ...  ...    ...   
164   Nasrudin  Nasrudinov  Light Heavyweight    9      0   
165   Vladimir      Mineev       Middleweight   13      1   
166    Patrick         Mix       Bantamweight   13      0   
167      Soren         Bak        Lightweight   13      1   
168    Khusein     Khaliev        Lightweight   19      2   

             Fighting Style   Sex      Country                     Nickname  \
0                 Wrestling  Male          USA                   The Hammer   
1                Kickboxing  Male          USA                           Mo

In [27]:
df5.dtypes

First Name              object
Last Name               object
Weight Class            object
Wins                    object
Losses                  object
Fighting Style          object
Sex                     object
Country                 object
Nickname                object
Title Holder            object
Interim Title Holder    object
dtype: object

In [28]:
#EDA- test -- examine the current title holders with duplicated names showing either the ones who used to be Interim Champs or won the belt twice'

Current_champs = df5[df5['Title Holder'] == 'Current']
Current_champs

,First Name,Last Name,Weight Class,Wins,Losses,Fighting Style,Sex,Country,Nickname,Title Holder,Interim Title Holder
23,Stipe,Miocic,Heavyweight,20,3,Wresling,Male,USA,Unknown,Current,NaN
25,Stipe,Miocic,Heavyweight,20,3,Wresling,Male,USA,Unknown,Current,NaN
42,Jan,Blachowicz,Light Heavyweight,27,8,Jiu-Jitsu,Male,Poland,Prince of Cieszyn,Current,NaN
54,Israel,Adesanya,Middleweight,20,0,"Kickboxing, Boxing",Male,Nigeria,The Last Stylebender,Current,interim
55,Israel,Adesanya,Middleweight,20,0,"Kickboxing, Boxing",Male,NaN,The Last Stylebender,Current,NaN
70,Kamaru,Usman,Welterweight,17,1,Wrestling,Male,Nigeria,The Nigerian Nightmare,Current,NaN
81,Khabib,Nurmagomedov,Lightweight,29,0,Sambo,Male,Russia,The Eagle,Current,NaN
82,Khabib,Nurmagomedov,Lightweight,29,0,Sambo,Male,Russia,The Eagle,Current,NaN
101,Petr,Yan,Bantamweight,15,1,"Boxing, Muay thai",Male,Russia,No Mercy,Current,NaN
104,Deiveson,Figueiredo,Flyweight,20,1,Jiu-Jitsu,Male,Brazil,Daico,Current,NaN


In [29]:
#EDA_test

Current_champs['Title Holder'].value_counts

<bound method IndexOpsMixin.value_counts of 23     Current
25     Current
42     Current
54     Current
55     Current
70     Current
81     Current
82     Current
101    Current
104    Current
107    Current
111    Current
113    Current
118    Current
Name: Title Holder, dtype: object>